# 🎬 Sailing Agent Trajectory Visualizer

This notebook provides an interactive interface for visualizing sailing agent trajectories.

**Features:**
1. **Single Agent Visualization**: Watch your agent navigate through the windfield step-by-step
2. **Multi-Agent Race Mode**: Compare multiple agents racing on the same windfield! 🏁


In [10]:
# If you've modified visualization.py, run this cell to reload it
import importlib
import sys
if 'src.visualization' in sys.modules:
    import src.visualization
    importlib.reload(src.visualization)
    print("✅ Reloaded visualization module")


✅ Reloaded visualization module


## Setup


In [11]:
import sys
import os
import numpy as np
import pandas as pd
from IPython.display import display

# Add the src directory to the path
sys.path.append(os.path.abspath('../src'))
sys.path.append(os.path.abspath('..'))

# Import the evaluation and visualization tools
from src.test_agent_validity import validate_agent
from src.evaluation import evaluate_agent, visualize_trajectory
from src.visualization import visualize_race, print_race_summary, create_race_gif
from initial_windfields import get_initial_windfield, INITIAL_WINDFIELDS

# List available initial windfields
print("Available windfields:")
for windfield_name in sorted(INITIAL_WINDFIELDS.keys()):
    print(f"  - {windfield_name}")

print("\n✅ Setup complete!")


Available windfields:
  - simple_static
  - static_headwind
  - training_1
  - training_2
  - training_3

✅ Setup complete!


---
# Part 1: Single Agent Trajectory Viewer

Watch a single agent navigate through the windfield with an interactive slider.


## Configuration


In [12]:
#############################################
### CONFIGURE YOUR VISUALIZATION HERE ######
#############################################

# Agent to visualize
AGENT_PATH = "../src/agents/private_my_agent_lucie.py"
# Other options:
# AGENT_PATH = "../src/agents/private_my_agent_xavier.py"
# AGENT_PATH = "../src/agents/agent_naive.py"

# Windfield to use
WINDFIELD_NAME = "static_headwind"

# Seed for reproducibility
SEED = 1

# Maximum steps
MAX_STEPS = 200

#############################################

print(f"Agent: {AGENT_PATH}")
print(f"Windfield: {WINDFIELD_NAME}")
print(f"Seed: {SEED}")
print(f"Max steps: {MAX_STEPS}")


Agent: ../src/agents/private_my_agent_lucie.py
Windfield: static_headwind
Seed: 1
Max steps: 200


## Load and Run Agent


In [13]:
# Load agent
validation_results = validate_agent(AGENT_PATH)
if not validation_results['valid']:
    print("❌ Agent validation failed:")
    for error in validation_results['errors']:
        print(f"  - {error}")
    raise ValueError("Invalid agent")

AgentClass = validation_results['agent_class']
agent = AgentClass()
print(f"✅ Loaded agent: {AgentClass.__name__}")

# Get windfield configuration
initial_windfield = get_initial_windfield(WINDFIELD_NAME)
initial_windfield['env_params'] = {
    'wind_grid_density': 32,  # Show all 32x32 arrows
    'wind_arrow_scale': 120,  # Higher scale = shorter arrows (prevents overlap)
    'render_mode': "rgb_array",
    'show_full_trajectory': True # show full trajectory 
}

# Run evaluation with rendering
print(f"\n🎬 Running agent on {WINDFIELD_NAME} with seed {SEED}...")
results = evaluate_agent(
    agent=agent,
    initial_windfield=initial_windfield,
    seeds=SEED,
    max_horizon=MAX_STEPS,
    verbose=False,
    render=True,
    full_trajectory=True
)

# Display results
print(f"\n📊 Results:")
print(f"  Reward: {results['rewards'][0]:.2f}")
print(f"  Steps: {results['steps'][0]}")
print(f"  Success: {'✅' if results['individual_results'][0]['success'] else '❌'}")
print(f"  Frames captured: {len(results['frames'])}")


✅ Loaded agent: MyAgent

🎬 Running agent on static_headwind with seed 1...

📊 Results:
  Reward: 100.00
  Steps: 67
  Success: ✅
  Frames captured: 67


## Interactive Trajectory Viewer


In [ ]:
# Display interactive slider
visualize_trajectory(results, None, with_slider=True)


interactive(children=(IntSlider(value=0, description='Step:', max=66), Output()), _dom_classes=('widget-intera…

---
# Part 2: Multi-Agent Race Viewer 🏁

Watch multiple agents race against each other on the same windfield!

**Note:** All agents will face the exact same wind conditions (same windfield + seed).


## Configuration


In [6]:
#############################################
### CONFIGURE YOUR RACE HERE ###############
#############################################

# Agents to race (add as many as you want!)
RACE_AGENTS = [
    {"path": "../src/agents/private_my_agent_lucie.py", "name": "Lucie's Agent", "color": "purple"},
    {"path": "../src/agents/private_my_agent_xavier.py", "name": "Xavier's Agent", "color": "orange"},
    {"path": "../src/agents/private_agent_tacking.py", "name": "Tibo's Smart Tacking", "color": "cyan"},
    # Alternative: Advanced version with perfect predictions (tracks accumulator)
    # {"path": "../src/agents/private_agent_tacking_advanced.py", "name": "Tacking (Advanced)", "color": "cyan"},
]

# Race windfield
RACE_WINDFIELD = "static_headwind"

# Race seed
RACE_SEED = 1

# Maximum steps
RACE_MAX_STEPS = 200

#############################################

print(f"🏁 Race Configuration:")
print(f"  Windfield: {RACE_WINDFIELD}")
print(f"  Seed: {RACE_SEED}")
print(f"  Agents: {len(RACE_AGENTS)}")
for i, agent_info in enumerate(RACE_AGENTS):
    print(f"    {i+1}. {agent_info['name']} ({agent_info['color']})")


🏁 Race Configuration:
  Windfield: static_headwind
  Seed: 1
  Agents: 3
    1. Lucie's Agent (purple)
    2. Xavier's Agent (orange)
    3. Tibo's Smart Tacking (cyan)


## Run All Agents


In [7]:
# Load and run all agents
race_results = []

for agent_info in RACE_AGENTS:
    print(f"\n🤖 Loading {agent_info['name']}...")
    
    # Load agent
    validation_results = validate_agent(agent_info['path'])
    if not validation_results['valid']:
        print(f"  ❌ Validation failed for {agent_info['name']}")
        continue
    
    AgentClass = validation_results['agent_class']
    agent = AgentClass()
    
    # Get windfield (must be same for all agents)
    initial_windfield = get_initial_windfield(RACE_WINDFIELD)
    
    # Run evaluation (without rendering individual frames, we'll render the race custom)
    results = evaluate_agent(
        agent=agent,
        initial_windfield=initial_windfield,
        seeds=RACE_SEED,
        max_horizon=RACE_MAX_STEPS,
        verbose=False,
        render=False,  # We'll do custom rendering
        full_trajectory=True
    )
    
    # Store results with agent info
    race_results.append({
        'name': agent_info['name'],
        'color': agent_info['color'],
        'positions': results['positions'],
        'actions': results['actions'],
        'reward': results['rewards'][0],
        'steps': results['steps'][0],
        'success': results['individual_results'][0]['success']
    })
    
    print(f"  ✅ Reward: {results['rewards'][0]:.2f}, Steps: {results['steps'][0]}, Success: {results['individual_results'][0]['success']}")

print(f"\n✅ All {len(race_results)} agents completed!")



🤖 Loading Lucie's Agent...
  ✅ Reward: 100.00, Steps: 67, Success: True

🤖 Loading Xavier's Agent...
  ✅ Reward: 100.00, Steps: 51, Success: True

🤖 Loading Tibo's Smart Tacking...
  ✅ Reward: 100.00, Steps: 78, Success: True

✅ All 3 agents completed!


## Race Visualization

Use the slider to watch all agents move through the windfield simultaneously!


In [8]:
# Visualize the race using the visualization module!
# Visualize the race using the visualization module!
visualize_race(race_results, RACE_WINDFIELD, RACE_SEED, RACE_MAX_STEPS, 
               show_full_trajectories=True)  # ← Add this parameter

interactive(children=(IntSlider(value=0, description='Race Step:', max=77), Output()), _dom_classes=('widget-i…

## Race Summary


In [9]:
# Display race summary using the visualization module
print_race_summary(race_results)



🏆 RACE RESULTS 🏆


,Agent,Color,Steps,Reward,Success
1,Xavier's Agent,orange,51,100.00,✅
0,Lucie's Agent,purple,67,100.00,✅
2,Tibo's Smart Tacking,cyan,78,100.00,✅



🥇 WINNER: Xavier's Agent (completed in 51 steps!)


---
# Part 3: Create a GIF from the Race 🎬

Want to save the race as a shareable GIF animation? Use the cell below!

---
## Tips

**Single Agent Mode:**
- Use the slider to step through each frame of the trajectory
- Watch how the agent responds to wind conditions
- Check the info box for detailed state information
- Default agent is Lucie's Q-learning agent - try Xavier's recursive search agent too!

**Race Mode:**
- Add more agents to `RACE_AGENTS` list
- Try different windfields to see how agents adapt
- Look for the dashed trail behind each boat to see their path
- Gold stars (⭐) indicate successful completion!
- Compare Lucie's learned Q-table approach vs Xavier's look-ahead search

**Customization:**
- Change `WINDFIELD_NAME` to test different scenarios (`static_headwind`, `training_1`, `training_2`, `training_3`)
- Adjust `SEED` to see different random variations
- Modify colors in `RACE_AGENTS` for better visibility
- Try different `MAX_STEPS` values to see longer/shorter episodes

**Code Organization:**
- All visualization logic is now in `src/visualization.py`
- Easy to reuse in your own scripts
- Clean separation between evaluation and visualization

Happy sailing! ⛵


## Create GIF Animation

Run this cell to export the race as an animated GIF file!


In [9]:
#############################################
### GIF CONFIGURATION #######################
#############################################

# Enable/disable GIF creation
CREATE_GIF = True  # Set to True to create a GIF

# GIF settings
GIF_OUTPUT_PATH = "race_animation.gif"  # Where to save the GIF
GIF_FPS = 10  # Frames per second (higher = faster animation)
GIF_STEP_INTERVAL = 1  # 1 = every step, 2 = every other step (reduces file size)

#############################################

if CREATE_GIF:
    if 'race_results' not in locals():
        print("⚠️ Please run the race evaluation cells first!")
    else:
        create_race_gif(
            race_results=race_results,
            windfield_name=RACE_WINDFIELD,
            seed=RACE_SEED,
            output_path=GIF_OUTPUT_PATH,
            fps=GIF_FPS,
            step_interval=GIF_STEP_INTERVAL,
            show_full_trajectories=True  # ← Add this parameter
        )
else:
    print("ℹ️ GIF creation is disabled. Set CREATE_GIF = True to enable.")


🎬 Creating race GIF...


/Users/t.rahier/Desktop/code/RL_project_ENSAE/src/visualization.py:374: UserWarning: Setting the 'color' property will override the edgecolor or facecolor properties.
  boat_polygon = Polygon(boat_vertices, color=result['color'], alpha=0.9,


/Users/t.rahier/Desktop/code/RL_project_ENSAE/src/visualization.py:451: UserWarning: Glyph 9989 (\N{WHITE HEAVY CHECK MARK}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/Users/t.rahier/Desktop/code/RL_project_ENSAE/src/visualization.py:455: UserWarning: Glyph 9989 (\N{WHITE HEAVY CHECK MARK}) missing from font(s) DejaVu Sans.
  plt.savefig(buf, format='png', dpi=100, bbox_inches='tight')


  Processed 78/78 frames...
💾 Saving GIF to race_animation.gif...
✅ GIF created successfully! Saved to: race_animation.gif
   Total frames: 78 | Duration: ~7.8 seconds


**GIF Parameters:**
- `GIF_FPS`: Controls animation speed (10 = normal, 15 = faster, 5 = slower)
- `GIF_STEP_INTERVAL`: Sample every N steps (1 = all frames, 2 = half frames)
- Higher step interval = smaller file size but less smooth

**Note:** You need to install `imageio` to create GIFs:
```bash
pip install imageio
```
